<img align="right" width="125" src="https://www.ou.nl/documents/40554/3255217/Logo_OU.jpg"/>

<br>
<center> <font size ="6" color='red'> Production Planning Cockpit </font></center>


In [1]:
import os
online_version = False;editmode = True
if online_version:  
    user = "muratfirat78";repo = "ProductionPlanning"
    if os.path.isdir(repo):
        !rm -rf {repo}

    !git clone https://github.com/{user}/{repo}.git
    %cd /content/{repo}
from DBMain import*
DataMgr.setOnlineVersion(online_version);VisMgr.setEditMode(editmode)
#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); 
tab_5 = VisMgr.getProductionProgressTab().generatePPrgTAB()
tab_3 = VisMgr.generatePLTAB(); tab_6 = VisMgr.getSchedulingTab().generatePSschTAB()
tab_7 = VisMgr.getSimulationTab().generateSimTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_5,tab_3,tab_6,tab_7])


tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Instance Data');tab_set.set_title(2, 'Production Progress')
tab_set.set_title(3, 'Planning');tab_set.set_title(4, 'Scheduling');tab_set.set_title(5, 'Simulation')
tab_set